## Importing All Required Modules

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb

## Datasets

In [2]:
dataTrain = pd.read_csv("train.csv")
dataTest = pd.read_csv("test.csv")

In [3]:
dataTrain.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [4]:
dataTest.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [5]:
y_train = dataTrain['price']

In [6]:
dataTotal = pd.concat([dataTrain,dataTest])
dataTotal.index = dataTotal['id']

In [7]:
dataTotal.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,,
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200.0
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999.0
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900.0
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000.0
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500.0


In [8]:
data = dataTotal.copy()
data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,,
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200.0
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999.0
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900.0
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000.0
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500.0


In [9]:
dataTrain['transmission'].unique()

array(['A/T', 'Transmission w/Dual Shift Mode', '7-Speed A/T',
       '8-Speed A/T', '10-Speed Automatic', '1-Speed A/T', '6-Speed A/T',
       '10-Speed A/T', '9-Speed A/T', '8-Speed Automatic',
       '9-Speed Automatic', '5-Speed A/T', 'Automatic',
       '7-Speed Automatic with Auto-Shift', 'CVT Transmission',
       '5-Speed M/T', 'M/T', '6-Speed M/T', '6-Speed Automatic',
       '4-Speed Automatic', '7-Speed M/T', '2-Speed A/T',
       '1-Speed Automatic', 'Automatic CVT', '4-Speed A/T',
       '6-Speed Manual', 'Transmission Overdrive Switch',
       '8-Speed Automatic with Auto-Shift', '7-Speed Manual',
       '7-Speed Automatic', '9-Speed Automatic with Auto-Shift',
       '6-Speed Automatic with Auto-Shift',
       '6-Speed Electronically Controlled Automatic with O', 'F', 'CVT-F',
       '8-Speed Manual', 'Manual', '–', '2', '6 Speed At/Mt',
       '5-Speed Automatic', '2-Speed Automatic', '8-SPEED A/T', '7-Speed',
       'Variable', 'Single-Speed Fixed Gear', '8-SPEED AT',


## Extensive Preprocessing

### Dividing the "Transmission" Column

In [10]:
speedKeywords = ['Speed']
transmissionKeywords = ['A/T', 'Automatic', 'Manual', 'CVT', 'At/Mt', 'AT', 'Mt', 'M/T']
specialKeywords = ['Dual Shift Mode', 'Auto-Shift', 'Electronically Controlled', 'Overdrive']
otherKeywords = ['F', 'Variable', 'Single-Speed Fixed Gear']

def transmissionCleaner(transmissionColumn):
    speed = None
    transmission = None
    special = None
    other = None

    for i in speedKeywords:
        if i in transmissionColumn:
            speed = ' '.join(word for word in transmissionColumn.split() if 'Speed' in word)
            break
    
    for i in transmissionKeywords:
        if i in transmissionColumn:
            transmission = i
            break
    
    for i in specialKeywords:
        if i in transmissionColumn:
            special = i
            break
    
    if all(x is None for x in [speed, transmission, special, other]):
        if any(i in transmissionColumn for i in otherKeywords):
            other = transmissionColumn
        else:
            other = None
    return pd.Series([speed, transmission, special, other])

data[['transmissionSpeed', 'transmissionCategory', 'transmissionSpecial', 'transmissionOther']] = data['transmission'].apply(transmissionCleaner)
dataPre = data.copy()

In [11]:
dataPre.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,transmissionSpeed,transmissionCategory,transmissionSpecial,transmissionOther
id,,,,,,,,,,,,,,,,,
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200.0,None,A/T,None,None
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999.0,None,A/T,None,None
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900.0,None,A/T,None,None
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000.0,None,None,Dual Shift Mode,None
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500.0,7-Speed,A/T,None,None


In [12]:
print(dataPre['engine'])

id
0              172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel
1              252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel
2         320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...
3              420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel
4              208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel
                                ...                        
314218        362.0HP 3.0L V6 Cylinder Engine Gasoline Fuel
314219                                      3.0 Liter Turbo
314220        333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel
314221        333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel
314222         355.0HP 5.3L 8 Cylinder Engine Gasoline Fuel
Name: engine, Length: 314223, dtype: object


### Series of Cells to Divide "Engine" Column

In [13]:
pattern = r'(\d+\.\d+)HP'

def e_power(engine):
    if pd.notnull(engine): 
        match = re.search(pattern, engine)
        if match:
            return match.group() 
    return None 

dataPre["e_power"] = dataPre['engine'].apply(e_power)

In [14]:
pattern = r'(\d+\.\d+)L'
pattern_L = r'(\d+\.\d+)\s*Liter'

def e_litter(litter):
    if pd.notnull(litter):  
        match = re.search(pattern, litter)
        match_L = re.search(pattern_L, litter)
        if match:
            return match.group() 
        if match_L:
            return match_L.group()         
    return None 

dataPre["e_litter"] = dataPre['engine'].apply(e_litter)

In [15]:
pattern_C = r'L(.+)Cylinder'
pattern_V = r'V(\d{1,2})'
pattern_I = r'I(\d{1,2})'
pattern_H = r'H(\d{1,2})'
pattern_W = r'W(\d{1,2})'


def e_cylinder(cylinder):
    if pd.notnull(cylinder):
        match_C = re.search(pattern_C, cylinder)
        match_V = re.search(pattern_V, cylinder)
        match_I = re.search(pattern_I, cylinder)
        match_H = re.search(pattern_H, cylinder)
        match_W = re.search(pattern_W, cylinder)
        if match_C:
            return match_C.group() 
        elif cylinder =="12 Cylinder Engine":
            return "12 Cylinder"
        elif match_I:
            return match_I.group()
        elif match_V:
            return match_V.group()
        elif match_H:
            return match_H.group()
        elif match_W:
            return match_W.group()
        return None 

dataPre["e_cylinder"] = dataPre['engine'].apply(e_cylinder)

In [16]:
pattern = r'(\d{1,2})V'

def e_valve(valve):
    if pd.notnull(valve):  
        match = re.search(pattern, valve)
        if match:
            return match.group()
        return None 

dataPre["e_valve"] = dataPre['engine'].apply(e_valve)

In [17]:
def camshaft_posi(camshaft):
    if pd.notnull(camshaft):
        if "DOHC" in camshaft:
            return "DOHC"
        elif "SOHC" in camshaft:
            return "SOHC"
        elif "OHV" in camshaft:
            return "OHV"
    return None

dataPre['e_camshaft'] = dataPre['engine'].apply(camshaft_posi)

In [18]:
def fuel_supply_system(fuel_system):
    if pd.notnull(fuel_system):
        if "GDI" in fuel_system:
            return "GDI"
        elif "MPFI" in fuel_system:
            return "MPFI"
        elif "PDI" in fuel_system:
            return "PDI"
        elif "DDI" in fuel_system:
            return "DDI"
        elif "TFSI" in fuel_system:
            return "TFSI"
        elif "SIDI" in fuel_system:
            return "SIDI"
        elif "GTDI" in fuel_system:
            return "GTDI"
        elif "TSI" in fuel_system:
            return "TSI"
    return None

dataPre['e_fuel_supply_system'] = dataPre['engine'].apply(fuel_supply_system)


In [19]:
def fuel_type_category(description):
    if pd.notnull(description):
        if "Turbo" in description:
            if "Twin Turbo" in description:
                return "Twin Turbo"
            elif "Turbo Diesel" in description:
                return "Turbo Diesel"
            elif "Turbo Hybrid" in description:
                return "Turbo Hybrid"
            elif "Turbo Flexible Fuel" in description:
                return "Turbo Flexible Fuel"
            elif "Intercooled Turbo" in description:
                if "Diesel" in description:
                    return "Intercooled Turbo Diesel"
                return "Intercooled Turbo"
            return "Turbo"
        elif "Supercharged" in description:
            return "Supercharged"
        elif "SC" in description:
            return "Supercharged"
        elif "T/C" in description:
            return "Turbo"
        elif "Flexible Fuel" in description or "Engine Flex Fuel Capability" in description:
            return "Flexible Fuel"
        elif "Hybrid" in description:
            return "Hybrid"
    return None

dataPre['e_fuel_type'] = dataPre['engine'].apply(fuel_type_category)

In [20]:
pattern = r"^Engine$"

def engine_type_category(description):
    if pd.notnull(description):
        match = re.search(pattern, description)
        if "Engine Gasoline Fuel" in description:
            return "Engine Gasoline Fuel"
        elif match:
            return "Engine Gasoline Fuel"
        elif "Engine Diesel Fuel" in description:
            return "Engine Diesel Fuel"
        elif "Rotary engine Gasoline Fuel" in description:
            return "Rotary engine Gasoline Fuel"
        elif "SC ULEV" in description:
            return "SC ULEV"
        elif "I-VTEC" in description:
            return "I-VTEC"
    return None

dataPre['e_engine_type'] = dataPre['engine'].apply(engine_type_category)

In [21]:
def hybrid_type_category(description):
    if pd.notnull(description):
        if "Engine Gasoline/Mild Electric Hybrid" == description:
            return "Engine Gasoline/Mild Electric Hybrid"
        elif "Engine Gas/Electric Hybrid" == description:
            return "Engine Gas/Electric Hybrid"
        elif "Hybrid" == description:
            return "Hybrid"
        elif "Twin Hybrid" == description:
            return "Twin Hybrid"
        elif "Dual Motor - Standard" == description:
            return "Dual Motor - Standard"
    return None

dataPre['e_hybrid_type'] = dataPre['engine'].apply(hybrid_type_category)


In [22]:
def motor_battery_category(description):
    if pd.notnull(description):
        if "Dual AC Electric Motors" == description:
            return "Dual AC Electric Motors"
        elif "Electric Motor Electric Fuel System" == description:
            return "Electric Motor Electric Fuel System"
        elif "Electric Motor Hydrogen Fuel" == description:
            return "Electric Motor Hydrogen Fuel"
        elif "Battery Electric" == description:
            return "Battery Electric"
        elif "120 AH" == description:
            return "120 AH"
        elif "111.2Ah / FR 70kW / RR 160kW (6)" == description:
            return "111.2Ah / FR 70kW / RR 160kW (6)"        
    return None

dataPre['e_motor_battery'] = dataPre['engine'].apply(motor_battery_category)

In [23]:
def electric_motor(electric):
    if pd.notnull(electric):  
        electric_str = str(electric)
        if "Electric Motor Electric Fuel System" in electric_str:
            return "Electric Motor Electric Fuel System"
        elif "Dual AC Electric Motors" in electric_str:
            return "Dual AC Electric Motors"
        elif "Electric Motor Hydrogen Fuel" in electric_str:
            return "Electric Motor Hydrogen Fuel"
    return None  

dataPre["e_electric_motor"] = dataPre["e_power"].apply(electric_motor)



In [24]:
pattern = r'\b\d{1,4}\b'

def horsepower(hp):
    if pd.notnull(hp):  
        hp_str = str(hp)
        match = re.search(pattern, hp_str)
        if match:
            return float(match.group()) 
    return None

dataPre['horsepower'] = dataPre['e_power'].apply(horsepower)

In [25]:
dataPre.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,...,e_cylinder,e_valve,e_camshaft,e_fuel_supply_system,e_fuel_type,e_engine_type,e_hybrid_type,e_motor_battery,e_electric_motor,horsepower
id,,,,,,,,,,,,,,,,,,,,,
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,...,L 4 Cylinder,None,None,None,None,Engine Gasoline Fuel,None,None,None,172.0
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,...,L 8 Cylinder,None,None,None,None,Engine Gasoline Fuel,None,None,None,252.0
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,...,L 8 Cylinder,None,None,None,Flexible Fuel,None,None,None,None,320.0
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,...,L 8 Cylinder,None,None,None,None,Engine Gasoline Fuel,None,None,None,420.0
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,...,L 4 Cylinder,None,None,None,None,Engine Gasoline Fuel,None,None,None,208.0


### Adding the Country of the Manufacturer as an Additional Detail

In [26]:
brandToCountry = {
    'MINI': 'UK',
    'Lincoln': 'USA',
    'Chevrolet': 'USA',
    'Genesis': 'South Korea',
    'Mercedes-Benz': 'Germany',
    'Audi': 'Germany',
    'Ford': 'USA',
    'BMW': 'Germany',
    'Tesla': 'USA',
    'Cadillac': 'USA',
    'Land': 'UK', 
    'GMC': 'USA',
    'Toyota': 'Japan',
    'Hyundai': 'South Korea',
    'Volvo': 'Sweden',
    'Volkswagen': 'Germany',
    'Buick': 'USA',
    'Rivian': 'USA',
    'RAM': 'USA',
    'Hummer': 'USA',
    'Alfa': 'Italy',
    'INFINITI': 'Japan',
    'Jeep': 'USA',
    'Porsche': 'Germany',
    'McLaren': 'UK',
    'Honda': 'Japan',
    'Lexus': 'Japan',
    'Dodge': 'USA',
    'Nissan': 'Japan',
    'Jaguar': 'UK',
    'Acura': 'Japan',
    'Kia': 'South Korea',
    'Mitsubishi': 'Japan',
    'Rolls-Royce': 'UK',
    'Maserati': 'Italy',
    'Pontiac': 'USA',
    'Saturn': 'USA',
    'Bentley': 'UK',
    'Mazda': 'Japan',
    'Subaru': 'Japan',
    'Ferrari': 'Italy',
    'Aston': 'UK',
    'Lamborghini': 'Italy',
    'Chrysler': 'USA',
    'Lucid': 'USA',
    'Lotus': 'UK',
    'Scion': 'Japan',
    'smart': 'Germany',
    'Karma': 'USA',
    'Plymouth': 'USA',
    'Suzuki': 'Japan',
    'FIAT': 'Italy',
    'Saab': 'Sweden',
    'Bugatti': 'France',
    'Mercury': 'USA',
    'Polestar': 'Sweden',
    'Maybach': 'Germany'
}

dataPre['country'] = dataPre['brand'].map(brandToCountry)

In [27]:
dataPre.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,...,e_valve,e_camshaft,e_fuel_supply_system,e_fuel_type,e_engine_type,e_hybrid_type,e_motor_battery,e_electric_motor,horsepower,country
id,,,,,,,,,,,,,,,,,,,,,
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,...,None,None,None,None,Engine Gasoline Fuel,None,None,None,172.0,UK
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,...,None,None,None,None,Engine Gasoline Fuel,None,None,None,252.0,USA
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,...,None,None,None,Flexible Fuel,None,None,None,None,320.0,USA
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,...,None,None,None,None,Engine Gasoline Fuel,None,None,None,420.0,South Korea
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,...,None,None,None,None,Engine Gasoline Fuel,None,None,None,208.0,Germany


### Simplifying the Interior and Exterior Colour Column

In [28]:
colorCategories = {
    'White': ['White', 'Snowflake White Pearl Metallic', 'Platinum White Pearl', 'Iridescent Pearl Tricoat', 
                'Super White', 'Pearl White', 'Bianco Monocerus', 'Bianco Icarus Metallic', 'Frozen White', 
                'Crystal White Pearl', 'Oryx White Prl', 'Yulong White', 'Balloon White', 'Matte White', 
                'Frozen White', 'Indus Silver', 'Pristine White'],
    
    'Black': ['Black', 'Santorini Black Metallic', 'Ebony Twilight Metallic', 'Magnetite Black Metallic', 
                'Diamond Black', 'Black Clearcoat', 'Black Raven', 'Phantom Black Pearl Effect / Black Roof', 
                'Black Cherry', 'Black Forest Green', 'Crystal Black Pearl', 'Super Black', 'Magnetic Black',
                'Twilight Black', 'Onyx', 'Black Obsidian', 'Obsidian Black Metallic', 'Nero Noctis', 
                'Jet Black Mica', 'Midnight Black', 'Stellar Black Metallic', 'Graphite Grey', 'Majestic Black Pearl'],
    
    'Gray': ['Gray', 'Magnetite Gray Metallic', 'Chronos Gray Metallic', 'Granite Crystal Clearcoat Metallic', 
               'Granite Crystal Metallic Clearcoat', 'Silver Mist', 'Tungsten Metallic', 'Lunar Rock', 
               'Stormy Sea', 'Magnetic Gray Metallic', 'Quartzite Grey Metallic', 'Carpathian Grey', 
               'Daytona Gray', 'Typhoon Gray', 'Daytona Gray Pearl Effect', 'Graphite Grey', 'Stone Gray Metallic'],
    
    'Red': ['Red', 'Firecracker Red Clearcoat', 'Flame Red Clearcoat', 'Hyper Red', 'Tango Red Metallic', 
              'Delmonico Red Pearlcoat', 'Cajun Red Tintcoat', 'Radiant Red Metallic II', 'Ruby Red Metallic', 
              'Ruby Flare Pearl', 'Scarlet Ember', 'Redline Red', 'Rosso Mars Metallic', 'Rosso Corsa'],
    
    'Blue': ['Blue', 'Vega Blue', 'Phytonic Blue Metallic', 'Deep Crystal Blue Mica', 'Northsky Blue Metallic', 
               'Midnight Blue Metallic', 'Antimatter Blue Metallic', 'Lunar Blue Metallic', 'Deep Blue Metallic', 
               'Twilight Blue Metallic', 'Blue Caelum', 'Portofino Blue Metallic', 'Blue Reflex Mica'],
}

def colourCleaner(color):
    for category, colors in colorCategories.items():
        if color in colors:
            return category
    return 'Other'

dataPre['ExteriorColour'] = dataPre['ext_col'].apply(colourCleaner)


In [29]:
def interiorColourCleaner(color):
    color = color.lower()  
    if 'gray' in color or 'grey' in color or 'slate' in color:
        return 'Gray'
    elif 'beige' in color or 'tan' in color or 'camel' in color:
        return 'Beige'
    elif 'black' in color or 'ebony' in color or 'nero' in color or 'onyx' in color:
        return 'Black'
    elif 'white' in color or 'pearl' in color or 'ivory' in color or 'oyster' in color:
        return 'White'
    elif 'red' in color or 'garnet' in color or 'hotspur' in color or 'pimento' in color:
        return 'Red'
    elif 'blue' in color or 'navy' in color or 'cobalt' in color:
        return 'Blue'
    elif 'green' in color or 'agave' in color or 'cypress' in color:
        return 'Green'
    elif 'brown' in color or 'espresso' in color or 'chestnut' in color or 'walnut' in color:
        return 'Brown'
    elif 'yellow' in color or 'gold' in color or 'amber' in color:
        return 'Yellow'
    elif 'orange' in color:
        return 'Orange'
    elif 'silver' in color or 'platinum' in color:
        return 'Silver'
    else:
        return 'Other'

dataPre['InteriorColour'] = dataPre['int_col'].apply(interiorColourCleaner)

In [30]:
pattern = r'\d+'

def numberOfCylinder(cylinder):
    if pd.notnull(cylinder):
        cylinder_str = str(cylinder)
        match = re.search(pattern, cylinder_str)
        if match:
            return float(match.group())  
    return None 

dataPre['NumberOfCylinder'] = dataPre['e_cylinder'].apply(numberOfCylinder)

## Cleaning the Data

In [31]:
dataClean = dataPre.copy()

dataClean["clean_title"] = dataClean["clean_title"].fillna("No")
dataClean["accident"] = dataClean["accident"].fillna("Unknown")
dataClean["fuel_type"] = dataClean["fuel_type"].fillna("-")
dataClean["horsepower"] = dataClean["horsepower"].fillna(dataClean["horsepower"].mean())

dataClean.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,...,e_fuel_type,e_engine_type,e_hybrid_type,e_motor_battery,e_electric_motor,horsepower,country,ExteriorColour,InteriorColour,NumberOfCylinder
id,,,,,,,,,,,,,,,,,,,,,
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,...,None,Engine Gasoline Fuel,None,None,None,172.0,UK,Other,Gray,4.0
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,...,None,Engine Gasoline Fuel,None,None,None,252.0,USA,Other,Beige,8.0
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,...,Flexible Fuel,None,None,None,None,320.0,USA,Blue,Gray,8.0
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,...,None,Engine Gasoline Fuel,None,None,None,420.0,South Korea,Black,Black,8.0
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,...,None,Engine Gasoline Fuel,None,None,None,208.0,Germany,Black,Beige,4.0


In [32]:
dataClean.columns

Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price', 'transmissionSpeed', 'transmissionCategory',
       'transmissionSpecial', 'transmissionOther', 'e_power', 'e_litter',
       'e_cylinder', 'e_valve', 'e_camshaft', 'e_fuel_supply_system',
       'e_fuel_type', 'e_engine_type', 'e_hybrid_type', 'e_motor_battery',
       'e_electric_motor', 'horsepower', 'country', 'ExteriorColour',
       'InteriorColour', 'NumberOfCylinder'],
      dtype='object')

In [33]:
columnsWithNone = ['e_litter', 'e_cylinder', 'e_power', 'transmissionCategory', 'e_engine_type', 'transmissionSpeed', 'e_fuel_type', 'e_fuel_supply_system', 'e_camshaft', 'e_valve', 'transmissionSpecial', 'e_motor_battery', 'transmissionOther', 'e_hybrid_type', 'e_electric_motor']
for col in columnsWithNone:
    dataClean[col] = dataClean[col].fillna("Unknown")

dataClean['NumberOfCylinder'] = dataClean['NumberOfCylinder'].fillna(dataClean['NumberOfCylinder'].mean())

dataClean.drop(columns=['id','engine','transmission','ext_col','int_col','e_power','e_cylinder'],axis=1,inplace=True)

In [34]:
dataClean.head()

,brand,model,model_year,milage,fuel_type,accident,clean_title,price,transmissionSpeed,transmissionCategory,...,e_fuel_type,e_engine_type,e_hybrid_type,e_motor_battery,e_electric_motor,horsepower,country,ExteriorColour,InteriorColour,NumberOfCylinder
id,,,,,,,,,,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,None reported,Yes,4200.0,Unknown,A/T,...,Unknown,Engine Gasoline Fuel,Unknown,Unknown,Unknown,172.0,UK,Other,Gray,4.0
1,Lincoln,LS V8,2002,143250,Gasoline,At least 1 accident or damage reported,Yes,4999.0,Unknown,A/T,...,Unknown,Engine Gasoline Fuel,Unknown,Unknown,Unknown,252.0,USA,Other,Beige,8.0
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,None reported,Yes,13900.0,Unknown,A/T,...,Flexible Fuel,Unknown,Unknown,Unknown,Unknown,320.0,USA,Blue,Gray,8.0
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,None reported,Yes,45000.0,Unknown,Unknown,...,Unknown,Engine Gasoline Fuel,Unknown,Unknown,Unknown,420.0,South Korea,Black,Black,8.0
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,None reported,Yes,97500.0,7-Speed,A/T,...,Unknown,Engine Gasoline Fuel,Unknown,Unknown,Unknown,208.0,Germany,Black,Beige,4.0


## Encoding the Data

In [35]:
data = dataClean.copy()
dataPrice = data['price']
encoder = LabelEncoder()
for col in data.columns:
    if data[col].dtype == "object":
        data[col] = encoder.fit_transform(data[col])

In [36]:
data.head()

,brand,model,model_year,milage,fuel_type,accident,clean_title,price,transmissionSpeed,transmissionCategory,...,e_fuel_type,e_engine_type,e_hybrid_type,e_motor_battery,e_electric_motor,horsepower,country,ExteriorColour,InteriorColour,NumberOfCylinder
id,,,,,,,,,,,,,,,,,,,,,
0,31,495,2007,213000,3,1,1,4200.0,11,0,...,9,1,1,4,0,172.0,6,3,4,4.0
1,28,930,2002,143250,3,0,1,4999.0,11,0,...,9,1,1,4,0,252.0,7,3,0,8.0
2,9,1575,2002,136731,2,1,1,13900.0,11,0,...,0,5,1,4,0,320.0,7,1,4,8.0
3,16,758,2017,19500,3,1,1,45000.0,11,8,...,9,1,1,4,0,420.0,4,0,1,8.0
4,36,1077,2021,7388,3,1,1,97500.0,6,0,...,9,1,1,4,0,208.0,1,0,0,4.0


In [37]:
x = data.loc[data["price"].notna()]
y = x['price']
x.drop(columns='price', inplace =True, axis = 1)

C:\Users\Ashwin V\AppData\Local\Temp\ipykernel_47728\3442467161.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.drop(columns='price', inplace =True, axis = 1)


In [38]:
x.head()

,brand,model,model_year,milage,fuel_type,accident,clean_title,transmissionSpeed,transmissionCategory,transmissionSpecial,...,e_fuel_type,e_engine_type,e_hybrid_type,e_motor_battery,e_electric_motor,horsepower,country,ExteriorColour,InteriorColour,NumberOfCylinder
id,,,,,,,,,,,,,,,,,,,,,
0,31,495,2007,213000,3,1,1,11,0,4,...,9,1,1,4,0,172.0,6,3,4,4.0
1,28,930,2002,143250,3,0,1,11,0,4,...,9,1,1,4,0,252.0,7,3,0,8.0
2,9,1575,2002,136731,2,1,1,11,0,4,...,0,5,1,4,0,320.0,7,1,4,8.0
3,16,758,2017,19500,3,1,1,11,8,1,...,9,1,1,4,0,420.0,4,0,1,8.0
4,36,1077,2021,7388,3,1,1,6,0,4,...,9,1,1,4,0,208.0,1,0,0,4.0


In [39]:
test = data.loc[data['price'].isna()]
test.drop(columns='price',axis=1,inplace=True)

C:\Users\Ashwin V\AppData\Local\Temp\ipykernel_47728\2172178917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(columns='price',axis=1,inplace=True)


## Splitting the Data

In [40]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

## Model Training

In [41]:
lgbModel = lgb.LGBMRegressor(random_state=42)
gbrModel = GradientBoostingRegressor(random_state=42)
lassoModel = Lasso(random_state=42)

def crossVal(model, x, y):
    y_pred = cross_val_predict(model, x, y, cv=10)
    rmse = np.sqrt(mean_squared_error(y,y_pred))
    return y_pred, rmse

lgbPred, lgbRmse = crossVal(lgbModel,x_train,y_train)
gbrPred, gbrRmse = crossVal(gbrModel,x_train,y_train)
lassoPred, lassoRmse = crossVal(lassoModel,x_train,y_train)

print("LGB: ",lgbRmse)
print("GBR: ",gbrRmse)
print("Lasso: ",lassoRmse)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005724 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1029
[LightGBM] [Info] Number of data points in the train set: 135743, number of used features: 24
[LightGBM] [Info] Start training from score 43994.535431
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005414 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1030
[LightGBM] [Info] Number of data points in the train set: 135743, number of used features: 24
[LightGBM] [Info] Start training from score 43864.622493
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005211 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is

In [42]:
ensemblePred = (lgbPred + gbrPred + lassoPred) / 3
ensembleRmse = np.sqrt(mean_squared_error(y_train, ensemblePred))
print("Ensemble: ",ensembleRmse)

Ensemble:  74106.4986473342


In [43]:
lgbModel.fit(x_train,y_train)
gbrModel.fit(x_train,y_train)
lassoModel.fit(x_train,y_train)

lgbTestPred = lgbModel.predict(test)
gbrTestPred = gbrModel.predict(test)
lassoTestPred = lassoModel.predict(test)

ensembleTestPred = (lgbTestPred * 0.7) + (gbrTestPred * 0.2) + (lassoTestPred * 0.1)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005794 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1032
[LightGBM] [Info] Number of data points in the train set: 150826, number of used features: 24
[LightGBM] [Info] Start training from score 43890.785316


In [44]:
answers = pd.DataFrame({'id':dataTest['id'], 'price':ensembleTestPred})

answers.to_csv("answer2.csv", index=False)

In [45]:
answers

,id,price
0,188533,16844.010337
1,188534,74002.827379
2,188535,57628.903470
3,188536,32703.813428
4,188537,29781.884906
...,...,...
125685,314218,29733.055492
125686,314219,49900.130475
125687,314220,25417.324958
125688,314221,16503.750035
